# Clustering WT, RPM, and RPMA organoids for clonal tracing in vitro to in vivo
Modified from [ExtFig7_RPM_RPMA_Organoid_Allo_Final_Clean.ipynb](https://github.com/TGOliver-lab/Ireland_Basal_SCLC_2025/blob/main/Python_Code/ExtFig7_RPM_RPMA_Organoid_Allo_Final_Clean.ipynb)  

Related to Extended Data Fig. 7

In [ ]:
#Import other relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
# Read 
os.chdir('/work/asi16')

## 1. Read in WT tracheal-basal derived organoids

In [ ]:
orgs_nocre=sc.read_10x_mtx('TBO_Pool_NoCre_NotCellPlexed/042024_custom_count_TBOpoolNoCellPlex/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)

In [ ]:
# Add metadata ID's
orgs_nocre.obs['Genotype'] = 'WT'
orgs_nocre.obs['Model'] = 'Organoid'
orgs_nocre.obs['Cre'] = 'No_Cre'
orgs_nocre.obs['UnID'] = 'WT_Org_NoCre'
orgs_nocre.obs['Batch'] = 'Org_No_Cre'

In [ ]:
orgs_nocre

In [ ]:
# Subsample Orgs no cre to have a more comparable number to compare to transformed organoids
orgs_nocre_subset=sc.pp.subsample(orgs_nocre, n_obs=5000, copy=True)

## 2. Read in Cre-transformed RPM and RPMA "CellTagged pre-Cre" organoids

In [ ]:
rpm_org=sc.read_10x_mtx('TBO_Pool_CMV/per_sample_outs/RPM/count/sample_filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)
rpma_org=sc.read_10x_mtx('TBO_Pool_CMV/per_sample_outs/RPMA/count/sample_filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)


In [ ]:
# Add appropriate metadata to samples
rpm_org.obs['Genotype'] = 'RPM'
rpm_org.obs['Model'] = 'Organoid'
rpm_org.obs['Cre'] = 'Cre'
rpm_org.obs['UnID'] = 'RPM_Org_Cre'
rpm_org.obs['Batch'] = 'Org_Cre'

rpma_org.obs['Genotype'] = 'RPMA'
rpma_org.obs['Model'] = 'Organoid'
rpma_org.obs['Cre'] = 'Cre'
rpma_org.obs['UnID'] = 'RPMA_Org_Cre'
rpma_org.obs['Batch'] = 'Org_Cre'

In [ ]:
rpm_org

In [ ]:
rpma_org

## 2. Concatenate WT and transformed organoid datasets

In [ ]:
#Concatenate datasets
adata= orgs_nocre_subset.concatenate([rpm_org, rpma_org], 
                              index_unique=None, join="outer")

In [ ]:
adata.obs_names_make_unique()

In [ ]:
adata.obs.groupby(["UnID"]).apply(len)

## 3. Perform QC

In [ ]:
#QC filtering
adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

sc.pp.filter_cells(adata, min_genes=200)
#sc.pp.filter_genes(orgs_all, min_cells=3)

adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filter data by slicing anndata object
adata = adata[adata.obs.n_genes_by_counts < 7000, :]
adata = adata[adata.obs.n_genes_by_counts > 2000, :]
adata = adata[adata.obs.total_counts > 2000, :]
adata = adata[adata.obs.pct_counts_mito < 15, :]

In [ ]:
adata.obs.groupby(["UnID"]).apply(len)

## 4. ID HVG and generate SCVI model for clustering with batch correction

In [ ]:
#Prep for HVG and scvi
#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])
#Create layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm")

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
adata.var['mean_'] = np.array(adata.X.mean(0))[0]
adata.var['frac_zero'] = 1 - np.array((adata.X > 0).sum(0))[0] / adata.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata.var.mean_, adata.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata.varm['df_poisson']= df_poisson

adata_query = adata[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata.obsm["X_scVI_1.1"] = latent

## 5. Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata, use_rep="X_scVI_1.1")
sc.tl.umap(adata, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata, key_added="leiden_scVI_1.1", resolution=1)

## 6. Visualize clustering and QC metrics by UMAP

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Cre", cmap="cmo.matter", s=30, ax=ax, vmax="p99.99", frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Genotype", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Model", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

#Additional QC bar graphs
adata.obs['cluster'] = adata.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="pct_counts_mito", ax=ax)


In [ ]:
# Dot plot key cell type markers
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata,figsize=[28,10],
    var_names=more_types,
    groupby='leiden_scVI_1.1',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
# Exclude 14 as doublets
bad_clust=['14']

#Filter out bad clusters
to_keep=(~adata.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
adata2 = adata[to_keep].copy()

### From here, continue iterating through runs of scvi modeling until no clear low quality cell clusters or non-tumor cells are observed.
### Start back up at "set up and train scvi model" and run through subsetting out "bad clusters".
### Each time clusters are removed, model is run again to recluster.

## ITERATION 2
### Final iteration for Extended Data Fig. 7a

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata2,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
adata2.var['mean_'] = np.array(adata2.X.mean(0))[0]
adata2.var['frac_zero'] = 1 - np.array((adata2.X > 0).sum(0))[0] / adata2.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata2.var.mean_, adata2.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata2, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata2.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata2.varm['df_poisson']= df_poisson

adata_query = adata2[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata2.obsm["X_scVI_1.2"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata2, use_rep="X_scVI_1.2")
sc.tl.umap(adata2, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata2, key_added="leiden_scVI_1.2", resolution=0.5)


### Visualize UMAP and final leiden clusters as in Ext. Data Fig. 7a,b

In [ ]:
#Add cluster label
adata2.obs['cluster'] = adata2.obs["leiden_scVI_1.2"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata2.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata2.obs, x="cluster", y="pct_counts_mito", ax=ax)


In [ ]:
sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata2,figsize=[28,10],
    var_names=more_types,
    groupby='leiden_scVI_1.2',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
adata2.obs.groupby(["UnID"]).apply(len)

In [ ]:
# Reorder
lin = ('WT_Org_NoCre','RPM_Org_Cre','RPMA_Org_Cre')
adata2.obs['UnID'] = adata2.obs['UnID'].cat.reorder_categories(list(lin))

In [ ]:
adata2.obs.groupby(["UnID"]).apply(len)

### Determine cluster marker genes to aid in basal cell state determination and for supplementary tables

In [ ]:
#Generate signatures from these basal cells organoids and allografts
sc.pp.normalize_total(adata2)
sc.pp.log1p(adata2)

# Extract top 500 marker genes for leiden clusters from data 
sc.tl.rank_genes_groups(adata2,'leiden_scVI_1.2', method='wilcoxon', n_genes=500)

result = adata2.uns['rank_genes_groups']
groups = result['names'].dtype.names
markergenes=pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']})

markergenes.to_csv('030525_adata2_RPM_RPMA_WT_organoids_leiden_1.2_clustermarkers.csv' )

In [ ]:
# Assign basal cell states / basal heterogeneity 
sc.pp.pca(adata2, n_comps=50)

In [ ]:
sc.tl.rank_genes_groups(adata2, groupby="leiden_scVI_1.2", method="wilcoxon")

In [ ]:
# Run dendrogram explicitly with specific parameters
sc.tl.dendrogram(adata2, groupby="leiden_scVI_1.2", use_rep='X_pca', n_pcs=50)

In [ ]:
# Plot top 10 marker genes by cluster
sc.pl.rank_genes_groups_dotplot(adata2, n_genes=10,figsize=[30,3], standard_scale='var')

In [ ]:
# 4 is prolif basal
# 2 is hillock-like

### Apply basal state signatures from Lin et al, Nature, 2024 and Goldfarbmuren, Nat Comm, 2020

In [ ]:
## Apply signatures and look at enrichments 
hillock=pd.read_csv('../data/luminal_basal_hillock.csv')
luminal=hillock.Luminal_hillock
basal=hillock.Basal_hillock

luminal=luminal.squeeze().str.strip().to_list()
basal=basal.squeeze().str.strip().to_list()


In [ ]:
sc.tl.score_genes(adata2, luminal, score_name='Luminal_hillock')
sc.tl.score_genes(adata2, basal, score_name='Basal_hillock')


In [ ]:
## Apply signatures and look at enrichments 
ct=pd.read_csv('../data/Goldfarbmuren_CellSigs_mhomologs.csv')
ct

prolif_basal=ct.Proliferating_basal
proteo_basal=ct.Proteasomal_basal
diff_basal=ct.Differentiating_basal
krt8high=ct.KRT8high
mucus_sec=ct.Mucus_secretory
ciliated=ct.Ciliated
pnec=ct.PNEC
iono_tuft=ct.Iono_tuft
smg_basal=ct.SMG_basal
smg_sec=ct.SMG_secretory

prolif_basal=prolif_basal.squeeze().str.strip().to_list()
proteo_basal=proteo_basal.squeeze().str.strip().to_list()
diff_basal=diff_basal.squeeze().str.strip().to_list()
krt8high=krt8high.squeeze().str.strip().to_list()
mucus_sec=mucus_sec.squeeze().str.strip().to_list()
ciliated=ciliated.squeeze().str.strip().to_list()
pnec=pnec.squeeze().str.strip().to_list()
iono_tuft=iono_tuft.squeeze().str.strip().to_list()
smg_basal=smg_basal.squeeze().str.strip().to_list()
smg_sec=smg_sec.squeeze().str.strip().to_list()



In [ ]:
sc.tl.score_genes(adata2, prolif_basal, score_name='prolif_basal')
sc.tl.score_genes(adata2, proteo_basal, score_name='proteo_basal')
sc.tl.score_genes(adata2, diff_basal, score_name='diff_basal')
sc.tl.score_genes(adata2, krt8high, score_name='krt8high')
sc.tl.score_genes(adata2, mucus_sec, score_name='mucus_sec')
sc.tl.score_genes(adata2, ciliated, score_name='ciliated')
sc.tl.score_genes(adata2, pnec, score_name='pnec')
sc.tl.score_genes(adata2, iono_tuft, score_name='iono_tuft')
sc.tl.score_genes(adata2, smg_basal, score_name='smg_basal')
sc.tl.score_genes(adata2, smg_sec, score_name='smg_sec')

### Visualize basal state heterogeneity signatures for Ext. Data Fig. 7c

In [ ]:
sigs=["Basal_hillock","Luminal_hillock","prolif_basal","proteo_basal","diff_basal",
                           "krt8high","mucus_sec","ciliated","pnec","iono_tuft","smg_basal","smg_sec"]


In [ ]:
# Based on signature enrichment patterns and top DEGs...

# 0,6   is proteosomal basal
# 1,5 is diff basal
# 2 is luminal hillock/secretory
# 3,7 is hillock/krt8 intermediate
# 4 is prolif basal



In [ ]:
adata2.obs["CellType_Sig_Based"]=adata2.obs["leiden_scVI_1.2"]

In [ ]:
# Define based on sigs above and DEGS below

# Define the column in .obs and the mapping for replacement
column_to_modify = "CellType_Sig_Based"  # Replace with the actual column name in adata.obs
replacement_dict = {
    "0": "Proteosomal_Basal",
    "1": "Differentiating_Basal",
    "2": "Luminal_Hillock/Secretory",
    "3": "Krt8_Interm/Hillock",
    "4": "Proliferative_Basal",
    "5": "Differentiating_Basal",
    "6": "Proteosomal_Basal",
    "7": "Krt8_Interm/Hillock"
}

# Ensure the column exists
if column_to_modify in adata2.obs:
    # Replace values using the mapping
    adata2.obs[column_to_modify] = adata2.obs[column_to_modify].replace(replacement_dict)
    print("Replacement completed.")
else:
    print(f"Column '{column_to_modify}' not found in adata.obs.")



### Validate assigned states by looking at top cluster markers by state

In [ ]:
sc.tl.rank_genes_groups(adata2, groupby="CellType_Sig_Based", method="wilcoxon")

# Run dendrogram explicitly with specific parameters
sc.tl.dendrogram(adata2, groupby="CellType_Sig_Based", use_rep='X_pca', n_pcs=50)


In [ ]:
sc.pl.rank_genes_groups_dotplot(adata2, n_genes=15,figsize=[30,3], standard_scale='var', color_map="cmo.dense",)

In [ ]:
adata2.write_h5ad("030525_RPM_RPMA_WT_Organoids_forCellTagwStates.h5ad")

## End of analysis in Scanpy, move to Seurat in R for annotation of CellTag data and final analyses.
## To do this, convert resulting h5ad anndata object from this script as a Seurat object in R.